In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0818 17:25:47.946565  2168 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 17:25:47.952528  2168 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 17:25:47.954522  2168 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1996 1997 1998] TEST: [   7   10   12   21   23   32   42   47   51   54   63   70   77   83
   88  105  108  113  114  125  126  134  135  136  141  142  148  149
  150  158  164  183  191  192  195  201  206  211  217  219  224  233
  234  243  245  247  249  250  255  256  268  271  273  274  283  291
  295  300  302  305  307  319  326  328  337  340  349  375  376  378
  385  387  388  389  390  392  393  398  410  417  421  429  431  434
  438  441  443  449  459  463  464  465  470  472  478  480  482  485
  497  503  506  509  513  518  519  520  527  528  534  538  539  544
  554  559  560  563  569  571  578  583  584  588  591  592  603  615
  618  624  625  627  634  643  645  652  653  656  659  660  662  665
  673  679  688  694  702  704  706  720  721  723  729  738  739  743
  746  749  759  764  766  768  773  783  790  791  793  796  798  801
  806  807  812  817  819  830  833  836  839  840  845  849  865  869
  874  881  889  893  899  9

W0818 17:25:49.761718  2168 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 19s 12ms/sample - loss: 0.6173 - acc: 0.1405
Epoch 2/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5704 - acc: 0.1577
Epoch 3/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5431 - acc: 0.1692
Epoch 4/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5233 - acc: 0.1769
Epoch 5/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5155 - acc: 0.1803
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5094 - acc: 0.1831
Epoch 7/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5071 - acc: 0.1844
Epoch 8/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5021 - acc: 0.1865
Epoch 9/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4974 - acc: 0.1886
Epoch 10/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2455 - acc: 0.2745
Epoch 81/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2432 - acc: 0.2752
Epoch 82/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2413 - acc: 0.2760
Epoch 83/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2401 - acc: 0.2763
Epoch 84/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2399 - acc: 0.2762
Epoch 85/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2379 - acc: 0.2770
Epoch 86/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2355 - acc: 0.2779
Epoch 87/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2349 - acc: 0.2781
Epoch 88/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2334 - acc: 0.2787
Epoch 89/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.23

...

[[   49   246    42   187     3   489    73   130]
 [    3 13371   260  2876    21  3002   285   924]
 [    4   437  1143   975    16   715   109   561]
 [    1  2212   555 26703   105  2315   262  1758]
 [    0    46    17   341   182    27     7    33]
 [    6  3119   568  3149    20 10672  1066  2146]
 [    2  1024   305  1343    12  3230  1515  1395]
 [    2  1054   515  2412    21  2255   563  4626]]
TRAIN: [   0    2    4 ... 1995 1997 1999] TEST: [   1    3    9   15   17   20   24   29   30   34   37   40   41   44
   46   53   67   68   74   84   85   98  107  111  120  130  132  143
  147  151  155  160  173  178  179  187  198  202  204  205  208  215
  220  221  225  226  240  244  246  251  257  276  285  303  314  315
  329  339  342  350  351  357  363  366  379  380  386  401  408  419
  422  425  428  433  436  437  445  450  457  462  471  477  479  483
  488  492  498  499  502  507  516  521  523  524  525  529  530  540
  542  543  551  557  562  565  568  572  577

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2870 - acc: 0.2568
Epoch 57/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2815 - acc: 0.2586
Epoch 58/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2790 - acc: 0.2599
Epoch 59/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2770 - acc: 0.2603
Epoch 60/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2760 - acc: 0.2606
Epoch 61/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2736 - acc: 0.2615
Epoch 62/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2703 - acc: 0.2626
Epoch 63/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2681 - acc: 0.2631
Epoch 64/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2676 - acc: 0.2635
Epoch 65/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.26

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5269 - acc: 0.1759
Epoch 5/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5175 - acc: 0.1803
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5106 - acc: 0.1833
Epoch 7/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5057 - acc: 0.1853
Epoch 8/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5047 - acc: 0.1862
Epoch 9/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4977 - acc: 0.1887
Epoch 10/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4944 - acc: 0.1903
Epoch 11/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4918 - acc: 0.1912
Epoch 12/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4869 - acc: 0.1931
Epoch 13/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.4855 - 

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2390 - acc: 0.2774
Epoch 84/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2399 - acc: 0.2771
Epoch 85/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2375 - acc: 0.2780
Epoch 86/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2366 - acc: 0.2784
Epoch 87/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2368 - acc: 0.2784
Epoch 88/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2345 - acc: 0.2790
Epoch 89/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2338 - acc: 0.2794
Epoch 90/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2315 - acc: 0.2800
Epoch 91/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2307 - acc: 0.2803
Epoch 92/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.22

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3759 - acc: 0.2307
Epoch 32/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3701 - acc: 0.2329
Epoch 33/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3658 - acc: 0.2337
Epoch 34/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3596 - acc: 0.2358
Epoch 35/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3541 - acc: 0.2377
Epoch 36/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3518 - acc: 0.2383
Epoch 37/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3452 - acc: 0.2402
Epoch 38/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3412 - acc: 0.2416
Epoch 39/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3387 - acc: 0.2424
Epoch 40/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.33

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_24 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_24 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_25 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_26 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 700, 200)         

In [10]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.3754708295099102, 0.3912221198140546, 0.39211172261178406, 0.38188467744489263, 0.39663500158079695]
0.38746487019228765
Precision total
[0.5464038536010872, 0.5619392467641846, 0.556405114533617, 0.5520270768973159, 0.5646524426108128]
0.5562855468814035
Recalls total
[0.5553966354339979, 0.573971725530762, 0.5662247284248643, 0.5616212718395199, 0.5717695489157745]
0.5657967820289836
F1 total
[0.5438318811152377, 0.559247163044368, 0.5552144438997556, 0.547937538565673, 0.5602676941300178]
0.5532997441510105
